In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/Othercomputers/Il mio MacBook Pro/real-time-style-transfer/chainer-fast-neuralstyle
path = "/content/drive/Othercomputers/Il mio MacBook Pro/real-time-style-transfer/chainer-fast-neuralstyle"

In [ ]:
!pip install chainer
!sh setup_model.sh

In [ ]:
import os
if not os.path.exists('train2014.zip'):
  !wget http://images.cocodataset.org/zips/train2014.zip

In [ ]:
%ls

In [ ]:
#!unzip 'train2014.zip' -d './coco'

In [ ]:
#!python train.py -s './sample_images/test_style.png' -g 0 -d './coco/train2014'

In [ ]:
from __future__ import print_function, division
import numpy as np
import os, re
import argparse
import random
from PIL import Image

from chainer import cuda, Variable, optimizers, serializers
from net import *

def load_image(path, size):
    image = Image.open(path).convert('RGB')
    w,h = image.size
    if w < h:
        if w < size:
            image = image.resize((size, size*h//w))
            w, h = image.size
    else:
        if h < size:
            image = image.resize((size*w//h, size))
            w, h = image.size
    image = image.crop(((w-size)*0.5, (h-size)*0.5, (w+size)*0.5, (h+size)*0.5))
    return xp.asarray(image, dtype=np.float32).transpose(2, 0, 1)

def gram_matrix(y):
    b, ch, h, w = y.data.shape
    features = F.reshape(y, (b, ch, w*h))
    gram = F.batch_matmul(features, features, transb=True)/np.float32(ch*w*h)
    return gram

def total_variation(x):
    xp = cuda.get_array_module(x.data)
    b, ch, h, w = x.data.shape
    wh = Variable(xp.asarray([[[[1], [-1]], [[0], [0]], [[0], [0]]], [[[0], [0]], [[1], [-1]], [[0], [0]]], [[[0], [0]], [[0], [0]], [[1], [-1]]]], dtype=np.float32))
    ww = Variable(xp.asarray([[[[1, -1]], [[0, 0]], [[0, 0]]], [[[0, 0]], [[1, -1]], [[0, 0]]], [[[0, 0]], [[0, 0]], [[1, -1]]]], dtype=np.float32))
    return F.sum(F.convolution_2d(x, W=wh) ** 2) + F.sum(F.convolution_2d(x, W=ww) ** 2)

In [ ]:
batchsize = 7
style_image = './sample_images/test_style.png'
image_size = 256
n_epoch = 2
lambda_tv = 1e-6
lambda_f = 1.0
lambda_s = 5.0
lambda_noise = 1000.0
noise_range = 30
noise_count = 1000
style_prefix, _ = os.path.splitext(os.path.basename(style_image))
output = style_prefix
dataset = './coco/train2014'

In [ ]:
if os.path.exists('fs.list'):
  # leggo da qui i path che se no ci mette un anno
  with open('fs.list') as f:
    imagepaths = f.read().splitlines()
else:
  # devo creare la lista leggendo dalla cartella
  fs = os.listdir(dataset)
  imagepaths = []
  for fn in fs:
      base, ext = os.path.splitext(fn)
      if ext == '.jpg' or ext == '.png':
          imagepath = os.path.join(dataset, fn)
          imagepaths.append(imagepath)
  with open('fs.list','w') as tfile:
    tfile.write('\n'.join(imagepaths))

#fs

In [ ]:
n_data = len(imagepaths)
print('num traning images:', n_data)
n_iter = n_data // batchsize
print(n_iter, 'iterations,', n_epoch, 'epochs')
#82783

In [ ]:
import glob

def check_resume(style_name, higher=False):
  higher_it = 1 if higher else 0

  query_model = glob.glob(f'./models/check_{style_name}*.model')
  if(query_model):
    assert len(query_model) == 2, "dunno where to start, 1 or more than 2 checkpoints for the same model found"
    query_model.sort()
    info = query_model[higher_it].split("/")[-1].split(".")[0].split("_")
    start_it_model = int(info[-1]) // batchsize
    start_epoch_model = int(info[-2])

  query_state = glob.glob(f'./models/{style_name}*.state')
  if(query_state):
    assert len(query_state) == 2, "dunno where to start, 1 or more than 2 checkpoints for the same state found"
    query_state.sort()
    info = query_state[higher_it].split("/")[-1].split(".")[0].split("_")
    start_it_state = int(info[-1]) // batchsize
    start_epoch_state = int(info[-2])

  if not query_model or not query_state:
    print("unmatching state and model found, starting over")
    return None, None, 0, 0

  if (start_epoch_model == start_epoch_state and start_it_model == start_it_state):
    initmodel = query_model[higher_it]
    resume = query_state[higher_it]
    return initmodel, resume, start_it_model, start_epoch_model

check_resume(output, path)

In [ ]:
checkpoint = 5
initmodel, resume, start_iteration, start_epoch = check_resume(output, path, True)

gpu_code = 0
lr = 1e-3

model = FastStyleNet()
vgg = VGG()
serializers.load_npz('vgg16.model', vgg)

if initmodel:
    print('load model from', initmodel)
    serializers.load_npz(initmodel, model)

if gpu_code >= 0:
    cuda.get_device(gpu_code).use()
    model.to_gpu()
    vgg.to_gpu()
xp = np if gpu_code < 0 else cuda.cupy

O = optimizers.Adam(alpha=lr)
O.setup(model)
if resume:
    print('load optimizer state from', resume)
    serializers.load_npz(resume, O)

style = vgg.preprocess(np.asarray(Image.open(style_image).convert('RGB').resize((image_size,image_size)), dtype=np.float32))
style = xp.asarray(style, dtype=xp.float32)
style_b = xp.zeros((batchsize,) + style.shape, dtype=xp.float32)
for i in range(batchsize):
    style_b[i] = style
feature_s = vgg(Variable(style_b))
gram_s = [gram_matrix(y) for y in feature_s]

for epoch in range(start_epoch, n_epoch):
    print('epoch', epoch)

    if noise_count:
        noiseimg = xp.zeros((3, image_size, image_size), dtype=xp.float32)

        # prepare a noise image
        for ii in range(noise_count):
            xx = random.randrange(image_size)
            yy = random.randrange(image_size)

            noiseimg[0][yy][xx] += random.randrange(-noise_range, noise_range)
            noiseimg[1][yy][xx] += random.randrange(-noise_range, noise_range)
            noiseimg[2][yy][xx] += random.randrange(-noise_range, noise_range)

    slack = checkpoint*2 # 2 save only the last 2 checkpoints
    for i in range(start_iteration, n_iter):
        model.zerograds()
        vgg.zerograds()

        indices = range(i * batchsize, (i+1) * batchsize)
        x = xp.zeros((batchsize, 3, image_size, image_size), dtype=xp.float32)
        for j in range(batchsize):
            x[j] = load_image(imagepaths[i*batchsize + j], image_size)

        xc = Variable(x.copy())

        if noise_count:
            # add the noise image to the source image
            noisy_x = x.copy()
            noisy_x = noisy_x + noiseimg

            noisy_x = Variable(noisy_x)
            noisy_y = model(noisy_x)
            noisy_y -= 120

        x = Variable(x)

        y = model(x)

        xc -= 120
        y -= 120

        feature = vgg(xc)
        feature_hat = vgg(y)

        L_feat = lambda_f * F.mean_squared_error(Variable(feature[2].data), feature_hat[2]) # compute for only the output of layer conv3_3

        L_style = Variable(xp.zeros((), dtype=np.float32))
        for f, f_hat, g_s in zip(feature, feature_hat, gram_s):
            L_style += lambda_s * F.mean_squared_error(gram_matrix(f_hat), Variable(g_s.data))

        L_tv = lambda_tv * total_variation(y)

        # the 'popping' noise is the difference in resulting stylizations
        # from two images that are very similar. Minimizing it results
        # in a much more stable stylization that can be applied to video.
        # Small changes in the input result in small changes in the output.
        if noise_count:
            L_pop = lambda_noise * F.mean_squared_error(y, noisy_y)
            L = L_feat + L_style + L_tv + L_pop
            print('Epoch {},{}/{}. Total loss: {}. Loss distribution: feat {}, style {}, tv {}, pop {}'
                         .format(epoch, i, n_iter, L.data,
                                 L_feat.data/L.data, L_style.data/L.data,
                                 L_tv.data/L.data, L_pop.data/L.data))
        else:
            L = L_feat + L_style + L_tv
            print('Epoch {},{}/{}. Total loss: {}. Loss distribution: feat {}, style {}, tv {}'
                         .format(epoch, i, n_iter, L.data,
                                 L_feat.data/L.data, L_style.data/L.data,
                                 L_tv.data/L.data))

        L.backward()
        O.update()

        if checkpoint > 0 and i % checkpoint == 0:
            if i >= start_iteration + slack:
                os.remove('models/check_{}_{}_{}.model'.format(output, epoch, i - slack))
                os.remove('models/check_{}_{}_{}.state'.format(output, epoch, i - slack))
            serializers.save_npz('models/check_{}_{}_{}.model'.format(output, epoch, i), model)
            serializers.save_npz('models/check_{}_{}_{}.state'.format(output, epoch, i), O)

    #print('save "style.model"')
    #serializers.save_npz('models/{}_{}.model'.format(output, epoch), model)
    #serializers.save_npz('models/{}_{}.state'.format(output, epoch), O)

serializers.save_npz('models/{}.model'.format(output), model)
serializers.save_npz('models/{}.state'.format(output), O)